In [1]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# -*- coding: utf-8 -*-
"""
Demo code

@author: Denis Tome'

"""
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
from base import SetType
import multiprocessing
from mocap import Mocap
from utils import config, ConsoleLogger
from utils import evaluate, io
from model import TempModel
from model_xregopose import xR_egoposemodel
from model_mo2cap2 import mo2cap_ours
from tqdm import tqdm
import os

LOGGER = ConsoleLogger("Main")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def main(model_path=None,output_path=None):
	"""Main"""

	LOGGER.info('Starting demo...')

	# ------------------- Data loader -------------------


	# let's load data from validation set as example
	data = Mocap(
		'/dataset/data/Dataset/TestSet',
		SetType.VAL,
		)
	data_loader = DataLoader(
		data,
		batch_size=16,
		num_workers=(multiprocessing.cpu_count() // 3) * 2,
		shuffle=True)

	# ------------------- Evaluation -------------------

	eval_body = evaluate.EvalBody()
	eval_upper = evaluate.EvalUpperBody()
	eval_lower = evaluate.EvalLowerBody()

	# ------------------- Read dataset frames -------------------

	model = TempModel(pretrained_path=model_path).to(device)
	model = torch.nn.DataParallel(model,device_ids=[0,1])
	model.eval()

	for it, test_batch in enumerate(tqdm(data_loader)):
		image = test_batch['image'].cuda()
		image_zoom = test_batch['image_zoom'].cuda()
		p3d = test_batch['joints_3d_cam']
		action = test_batch['action']
		input_img = {
			'image':image,
			# 'image_zoom' : image_zoom,
			}
		# LOGGER.info('Iteration: {}'.format(it))
		# LOGGER.info('Images: {}'.format(img.shape))
		# LOGGER.info('p2ds: {}'.format(p2d.shape))
		# LOGGER.info('p3ds: {}'.format(p3d.shape))
		# LOGGER.info('Actions: {}'.format(action))

		# -----------------------------------------------------------
		pred_output = model(input_img)
		# -----------------------------------------------------------

		# TODO: replace p3d_hat with model preditions
		p3d_hat = pred_output['pred_pose']

		# Evaluate results using different evaluation metrices
		y_output = p3d_hat.data.cpu().numpy() * 1000
		y_target = p3d.data.cpu().numpy() * 1000

		eval_body.eval(y_output, y_target, action)
		eval_upper.eval(y_output, y_target, action)
		eval_lower.eval(y_output, y_target, action)
		if (it+1)%200==0:
			print({'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()})
			# break
		# TODO: remove break
		

	# ------------------- Save results -------------------

	LOGGER.info('Saving evaluation results...')
	res = {'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()}

	io.write_json(output_path, res)

	LOGGER.info('Done.')

In [3]:
import glob
path_ =sorted(glob.glob('/workspace/2d_to_3d/apps/exp815'+'/*'))
path_

['/workspace/2d_to_3d/apps/exp815/0.pth',
 '/workspace/2d_to_3d/apps/exp815/1.pth',
 '/workspace/2d_to_3d/apps/exp815/2.pth',
 '/workspace/2d_to_3d/apps/exp815/3.pth',
 '/workspace/2d_to_3d/apps/exp815/4.pth',
 '/workspace/2d_to_3d/apps/exp815/5.pth',
 '/workspace/2d_to_3d/apps/exp815/6.pth',
 '/workspace/2d_to_3d/apps/exp815/7.pth',
 '/workspace/2d_to_3d/apps/exp815/8.pth',
 '/workspace/2d_to_3d/apps/exp815/9.pth',
 '/workspace/2d_to_3d/apps/exp815/best.pth']

In [4]:
import gc
gc.collect()
torch.cuda.empty_cache()
raise

RuntimeError: No active exception to reraise

In [5]:
for i,p in enumerate(path_):
	_path = f'/workspace/2d_to_3d/apps/eval/Ours_sil_dep_fusion(loss fix)815'
	_name = f'baseline815_{i}.json'
	if not os.path.exists(_path):
		os.makedirs(_path)

	main(model_path=p,output_path=os.path.join(_path,_name))

07:26:59 [INFO] Main: Starting demo...


  0%|          | 0/5352 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/con

{'FullBody': {'All': 156.1856689453125, 'UpperStretching': 162.35499572753906, 'LowerStretching': 156.10369873046875, 'Walking': 155.14199829101562, 'Patting': 145.05613708496094, 'Talking': 116.3424072265625, 'Gaming': 147.26654052734375, 'Reacting': 120.13792419433594, 'Gesticuling': 107.52223205566406, 'Greeting': 104.41271209716797}, 'UpperBody': {'All': 95.9258041381836, 'UpperStretching': 105.11554718017578, 'LowerStretching': 76.43011474609375, 'Walking': 89.29006958007812, 'Patting': 84.80149841308594, 'Talking': 86.93668365478516, 'Gaming': 95.88090515136719, 'Reacting': 85.75531768798828, 'Gesticuling': 76.2370834350586, 'Greeting': 71.77652740478516}, 'LowerBody': {'All': 216.4455108642578, 'UpperStretching': 219.59442138671875, 'LowerStretching': 235.7772674560547, 'Walking': 220.99392700195312, 'Patting': 205.31076049804688, 'Talking': 145.7481231689453, 'Gaming': 198.65220642089844, 'Reacting': 154.52053833007812, 'Gesticuling': 138.807373046875, 'Greeting': 137.048873901

  7%|▋         | 401/5352 [01:02<10:28,  7.88it/s]

{'FullBody': {'All': 154.4817657470703, 'UpperStretching': 160.00048828125, 'LowerStretching': 154.02035522460938, 'Walking': 153.32728576660156, 'Patting': 158.1243438720703, 'Talking': 112.94690704345703, 'Gaming': 146.4390411376953, 'Reacting': 120.592529296875, 'Gesticuling': 111.27742004394531, 'Greeting': 105.92991638183594}, 'UpperBody': {'All': 95.37232208251953, 'UpperStretching': 104.13041687011719, 'LowerStretching': 75.26216888427734, 'Walking': 89.0155258178711, 'Patting': 93.86146545410156, 'Talking': 85.83223724365234, 'Gaming': 97.22109985351562, 'Reacting': 85.86003112792969, 'Gesticuling': 77.72513580322266, 'Greeting': 75.49223327636719}, 'LowerBody': {'All': 213.59121704101562, 'UpperStretching': 215.8705596923828, 'LowerStretching': 232.77853393554688, 'Walking': 217.63902282714844, 'Patting': 222.38726806640625, 'Talking': 140.06155395507812, 'Gaming': 195.65701293945312, 'Reacting': 155.3250274658203, 'Gesticuling': 144.82969665527344, 'Greeting': 136.36759948730

...

 93%|█████████▎| 5001/5352 [11:22<00:51,  6.83it/s]

{'FullBody': {'All': 73.14616394042969, 'UpperStretching': 75.78439331054688, 'Gaming': 63.41817855834961, 'Walking': 72.76182556152344, 'LowerStretching': 70.96996307373047, 'Patting': 79.3404769897461, 'Reacting': 66.81663513183594, 'Talking': 52.251136779785156, 'Gesticuling': 58.71678924560547, 'Greeting': 49.33177185058594}, 'UpperBody': {'All': 55.443092346191406, 'UpperStretching': 61.080711364746094, 'Gaming': 51.42235565185547, 'Walking': 49.60410690307617, 'LowerStretching': 43.40289306640625, 'Patting': 55.799198150634766, 'Reacting': 54.940303802490234, 'Talking': 45.492820739746094, 'Gesticuling': 48.19867706298828, 'Greeting': 42.34133529663086}, 'LowerBody': {'All': 90.8492431640625, 'UpperStretching': 90.48807525634766, 'Gaming': 75.41400146484375, 'Walking': 95.9195327758789, 'LowerStretching': 98.53704071044922, 'Patting': 102.88175201416016, 'Reacting': 78.69296264648438, 'Talking': 59.00944900512695, 'Gesticuling': 69.23490142822266, 'Greeting': 56.32220458984375}}


 97%|█████████▋| 5201/5352 [11:50<00:21,  7.00it/s]

{'FullBody': {'All': 73.15263366699219, 'UpperStretching': 75.82125091552734, 'Gaming': 63.41297912597656, 'Walking': 72.7975082397461, 'LowerStretching': 70.88463592529297, 'Patting': 79.34151458740234, 'Reacting': 66.8078384399414, 'Talking': 52.25999450683594, 'Gesticuling': 58.613094329833984, 'Greeting': 49.26126480102539}, 'UpperBody': {'All': 55.457489013671875, 'UpperStretching': 61.12333679199219, 'Gaming': 51.42076873779297, 'Walking': 49.6028938293457, 'LowerStretching': 43.38581466674805, 'Patting': 55.80607223510742, 'Reacting': 54.88375473022461, 'Talking': 45.44548797607422, 'Gesticuling': 48.150970458984375, 'Greeting': 42.29200744628906}, 'LowerBody': {'All': 90.84778594970703, 'UpperStretching': 90.51915740966797, 'Gaming': 75.40518951416016, 'Walking': 95.99211883544922, 'LowerStretching': 98.38346862792969, 'Patting': 102.876953125, 'Reacting': 78.73192596435547, 'Talking': 59.074501037597656, 'Gesticuling': 69.07521057128906, 'Greeting': 56.23052215576172}}


100%|██████████| 5352/5352 [12:10<00:00,  7.33it/s]
09:41:19 [INFO] Main: Saving evaluation results...
09:41:19 [INFO] Main: Done.
